In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


In [3]:
import os
os.getcwd()


'C:\\Users\\DEBASRI PAL'

In [5]:
import os
os.getcwd()


'C:\\Users\\DEBASRI PAL'

In [6]:
import pandas as pd

df = pd.read_csv("train.csv")  # This now reads from current folder
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
import pandas as pd

# Load the dataset
df = pd.read_csv("train.csv")

# Check column names
print("Columns before dropping:")
print(df.columns)

# Drop columns that are not useful for prediction (only if they exist)
columns_to_drop = ["PassengerId", "Name", "Ticket", "Cabin"]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Fill missing 'Age' with mean
df["Age"] = df["Age"].fillna(df["Age"].mean())

# Fill missing 'Embarked' with mode
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])

# One-hot encode categorical variables: 'Sex' and 'Embarked'
df = pd.get_dummies(df, columns=["Sex", "Embarked"], drop_first=True)

# Show the cleaned DataFrame
df.head()


Columns before dropping:
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,True,False,True
1,1,1,38.0,1,0,71.2833,False,False,False
2,1,3,26.0,0,0,7.9250,False,False,True
3,1,1,35.0,1,0,53.1000,False,False,True
4,0,3,35.0,0,0,8.0500,True,False,True


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define input features and target variable
X = df.drop("Survived", axis=1)
y = df["Survived"]

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ----------------------
# Logistic Regression
# ----------------------
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)
y_pred_log = log_model.predict(X_test)

# ----------------------
# Random Forest
# ----------------------
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# ----------------------
# Support Vector Classifier (SVM)
# ----------------------
svm_model = SVC()
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# ----------------------
# Evaluation Function
# ----------------------
def evaluate_model(name, y_true, y_pred):
    print(f"\n🔹 {name} Results:")
    print(f"Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall:    {recall_score(y_true, y_pred):.4f}")
    print(f"F1-score:  {f1_score(y_true, y_pred):.4f}")

# ----------------------
# Evaluate all 3 models
# ----------------------
evaluate_model("Logistic Regression", y_test, y_pred_log)
evaluate_model("Random Forest", y_test, y_pred_rf)
evaluate_model("SVM", y_test, y_pred_svm)



🔹 Logistic Regression Results:
Accuracy:  0.8101
Precision: 0.7857
Recall:    0.7432
F1-score:  0.7639

🔹 Random Forest Results:
Accuracy:  0.8156
Precision: 0.7971
Recall:    0.7432
F1-score:  0.7692

🔹 SVM Results:
Accuracy:  0.6592
Precision: 0.7600
Recall:    0.2568
F1-score:  0.3838


In [12]:
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [4, 6, 8, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Setup RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(),
    param_distributions=param_grid,
    n_iter=10,
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit to training data
random_search.fit(X_train, y_train)

# Show best params
print("✅ Best Hyperparameters:", random_search.best_params_)

# Evaluate optimized model
best_rf = random_search.best_estimator_
y_pred_best = best_rf.predict(X_test)

evaluate_model("Random Forest (Tuned)", y_test, y_pred_best)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
✅ Best Hyperparameters: {'n_estimators': 150, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 6, 'bootstrap': True}

🔹 Random Forest (Tuned) Results:
Accuracy:  0.8101
Precision: 0.8226
Recall:    0.6892
F1-score:  0.7500


## ✅ Summary

We compared three ML models on the Titanic dataset:

- Logistic Regression  
- Random Forest  
- SVM  

Then, we tuned **Random Forest** using `RandomizedSearchCV`.  
The best hyperparameters found were:

```python
{
    'n_estimators': 150,
    'min_samples_split': 5,
    'min_samples_leaf': 1,
    'max_depth': 6,
    'bootstrap': True
}
